In [1]:
def read_many_hdf5():
  
  walks,counts = [],[]
  
  # Open the HDF5 file
  file = h5py.File('/content/drive/My Drive/CapstoneWalks/MetaWalks.h5', "r+")
  walks = np.array(file["/walks"]).astype("uint8")
  file.close()

  file = h5py.File('/content/drive/My Drive/CapstoneWalks/MetaCounts.h5', "r+")
  counts = np.array(file["/counts"]).astype("uint8")
  file.close()
  return walks,counts


def read_one_hdf5(path):
  
  data = []
  
  # Open the HDF5 file
  file = h5py.File(path, "r+")
  data = np.array(file["default"]).astype("float32")
  file.close()
  print("Completed")
  return data

def writeSingleDataset(arr,path):

  with h5py.File(path, 'w') as f:
    dset = f.create_dataset("default", (arr.shape[0],arr.shape[1],arr.shape[2]))
    dset[0:arr.shape[0]] = arr
    f.close()
  return print("completed")


def process_loaded_data(img_rows, img_cols, in_channel):
  #Read images from files
  walk_imgs,count_imgs = read_many_hdf5()
  targets = pd.read_csv('/content/drive/My Drive/CapstoneWalks/MetaTargets.csv')
  #Convert data type and scale 
  walk_imgs,count_imgs = np.asarray(walk_imgs, dtype=np.float32),np.asarray(count_imgs, dtype=np.float32)
  walk_imgs,count_imgs = walk_imgs/255,count_imgs/steps

  # reshape to match Keras expectaions
  walk_imgs = walk_imgs.reshape(walk_imgs.shape[0], img_rows, img_cols, in_channel)
  count_imgs = count_imgs.reshape(count_imgs.shape[0], img_rows, img_cols, in_channel)

  #We want to pair the images together, we have a dimension of 4
  x_train_Stack = np.stack((walk_imgs,count_imgs), axis=4)


  x_train, x_test, y_train, y_test = train_test_split(x_train_Stack, targets, test_size = 10000, random_state=42)


  x_train_Walks = np.asarray(x_train[:,:,:,:,0])
  x_test_Walks = np.asarray(x_test[:,:,:,:,0])


  x_train_Counts = np.asarray(x_train[:,:,:,:,1])
  x_test_Counts = np.asarray(x_test[:,:,:,:,1])

  return x_train_Walks, x_test_Walks, x_train_Counts, x_test_Counts, y_train, y_test

def store_many_hdf5(data1, data2, path):
  """ Stores an array of images to HDF5.
  Parameters:
  ---------------
  walks       images array, (N, 100, 100, 3) to be stored
  counts      images array, (N, 100, 100, 3) to be stored
  """
  # Create a new HDF5 file
  file = h5py.File(path, "w")
  # Create a dataset in the file
  data1 = file.create_dataset("data1", np.shape(data1), h5py.h5t.STD_U8BE, data=data1,)
  file.close()
  
  # Create a new HDF5 file
  file = h5py.File(path, "a")
  # Create a dataset in the file
  data2 = file.create_dataset("data2", np.shape(data2), h5py.h5t.STD_U8BE, data=data2)
  file.close()
  
  return print("Completed")

def CombineProcessedData(arr,arr2,path,y_true):
  if y_true == 1:
    with h5py.File(path, 'w') as f:
      dset = f.create_dataset("default", (2*arr.shape[0],arr.shape[1]))
      dset[0:arr.shape[0]] = arr
      print("Halfway")
      dset[arr.shape[0]:(2*arr.shape[0])] = arr2
      f.close()
    return print("completed")
  
  with h5py.File(path, 'w') as f:
    dset = f.create_dataset("default", (2*arr.shape[0],arr.shape[1],arr.shape[2]))
    dset[0:arr.shape[0]] = arr
    print("Halfway")
    dset[arr.shape[0]:(2*arr.shape[0])] = arr2
    f.close()
    return print("completed")